In [ ]:
!nvidia-smi

In [ ]:
#%pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12
#%pip install cucim-cu12 cupy-cuda12x

In [1]:

import pandas as pd
import tensorflow as tf

import glob
import re

2024-11-16 03:21:57.429816: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-16 03:21:57.461245: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-16 03:21:57.470869: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-16 03:21:57.493082: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
name_mapping = [
    "box",
    "circularTorus",
    "cone",
    "coneOffset",
    "cylinder",
    "cylinderSlope",
    "dish",
    "mesh",
    "pyramid",
    "rectangularTorus",
    "sphere"
]

In [ ]:
def sort_by_number(texts:list[str]):
    def key(text:str):
        text = re.sub(r'.*photos_', '', text)
        text = re.sub(r'\.csv', '', text)
        text = re.sub(r'\D', '', text)
        return int(text)
    return sorted(texts, key=key)

In [ ]:
%pip install fastparquet

In [4]:
base_path = '/home/workspace/geometry-classifier/data/'

#csv_files = sort_by_number(glob.glob(base_path+'photos_v3_parquet/*.parquet'))


In [ ]:
for csv_file in csv_files:
    print(csv_file)
    df = pd.read_csv(csv_file)
    df.to_parquet(re.sub(r'\.csv', '.parquet', csv_file), index=False)

In [14]:
!du -h /home/workspace/geometry-classifier/data/tfrecord

16M	/home/workspace/geometry-classifier/data/tfrecord


In [10]:
feature_description = {
    "pixels": tf.io.FixedLenFeature([], tf.string),  # Raw pixel bytes
    "label": tf.io.FixedLenFeature([], tf.string),  # Raw label byte
}

# Parse function for a single TFRecord example
def parse_example(example_proto):
    parsed = tf.io.parse_single_example(example_proto, feature_description)
    # Decode and reshape data
    pixels = tf.io.decode_raw(parsed["pixels"], tf.uint8)
    label = tf.io.decode_raw(parsed["label"], tf.uint8)
    pixels = tf.reshape(pixels, [50176])  # Adjust shape if necessary
    return pixels, label

# Generator function to create a dataset
def create_tfrecord_dataset(file_list, batch_size=32):
    # Use tf.data.Dataset.list_files for dynamic file handling (optional)
    dataset = tf.data.TFRecordDataset(file_list, compression_type="GZIP")
    dataset = dataset.map(parse_example, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)  # Optimize performance
    return dataset

# Create the dataset
batch_size = 32
dataset = create_tfrecord_dataset(glob.glob(base_path+"tfrecord/*.gz"), batch_size=batch_size)

In [11]:

for batch_pixels, batch_labels in dataset:
    print("Batch pixels shape:", batch_pixels.shape)
    print("Batch labels shape:", batch_labels.shape)
    break  # Only to show the first batch

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes at component 0: expected [?,50176] but got [32,401408]. [Op:IteratorGetNext] name: 